This is my first experiment in this competition. Whereas XGBoost is highly recommended I rather tried to see how far I can go with an NN (using Keras).

This is the basic model and with 250 epochs has an accuracy of 80% (really poor).

I'll continue for a few days researching how much I can optimize this model.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
print(os.listdir("../input/"))
print(os.listdir("../working/"))
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

['learn-together']
['__notebook_source__.ipynb', '.ipynb_checkpoints']
/kaggle/input/learn-together/sample_submission.csv
/kaggle/input/learn-together/test.csv
/kaggle/input/learn-together/train.csv


In [2]:
#load data, I had an issue with the data and hacked the data into the kernel
#dftrain=pd.read_csv('/kaggle/input/train.csv')
#dftest=pd.read_csv('/kaggle/input/test.csv')
dftrain=pd.read_csv('/kaggle/input/learn-together/train.csv')
dftest=pd.read_csv('/kaggle/input/learn-together/test.csv')

In [ ]:
#file shape
print(dftrain.head())
print(dftrain.shape[0])
print(dftest.head())
print(dftest.shape[0])

In [ ]:
#features types
print(dftrain.dtypes)
print(dftest.dtypes)

In [ ]:
#validate files - nans per feature
print(dftrain.isnull().sum(axis = 0))
print(dftest.isnull().sum(axis = 0))

In [ ]:
#validate files - no rows with all zeros
print(dftrain[dftrain.drop(['Id','Cover_Type'], axis=1).eq(0).all(1)].empty)
print(dftest[dftest.drop('Id', axis=1).eq(0).all(1)].empty)

In [3]:
#split train data in features and labels
y = dftrain.Cover_Type
x = dftrain.drop(['Id','Cover_Type'], axis=1)

# split test data in features and Ids
Ids = dftest.Id
x_predict = dftest.drop('Id', axis=1)

In [4]:
#force all types to float
x = x.astype(float)
x_predict = x_predict.astype(float)

In [5]:
#normalize features
#in the future it can be done more elegantly, for now just using the max min values of the data that we know
#x['Elevation']=(x['Elevation']-x['Elevation'].min())/(x['Elevation'].max()-x['Elevation'].min())                             
x['Elevation']=(x['Elevation']-1859)/(3858-1859)                             
x['Aspect']=x['Aspect']/360                      
x['Slope']=x['Slope']/66                      
x['Horizontal_Distance_To_Hydrology']=x['Horizontal_Distance_To_Hydrology']/1397                      
x['Vertical_Distance_To_Hydrology']=(x['Vertical_Distance_To_Hydrology']+173)/(601+173)                             
x['Horizontal_Distance_To_Roadways']=x['Horizontal_Distance_To_Roadways']/7117                      
x['Hillshade_9am']=x['Hillshade_9am']/254                      
x['Hillshade_Noon']=x['Hillshade_Noon']/254                      
x['Hillshade_3pm']=x['Hillshade_3pm']/254                      
x['Horizontal_Distance_To_Fire_Points']=x['Horizontal_Distance_To_Fire_Points']/67173                      
                                
x_predict['Elevation']=(x_predict['Elevation']-1859)/(3858-1859)                             
x_predict['Aspect']=x_predict['Aspect']/360                      
x_predict['Slope']=x_predict['Slope']/66                      
x_predict['Horizontal_Distance_To_Hydrology']=x_predict['Horizontal_Distance_To_Hydrology']/1397                      
x_predict['Vertical_Distance_To_Hydrology']=(x_predict['Vertical_Distance_To_Hydrology']+173)/(601+173)                             
x_predict['Horizontal_Distance_To_Roadways']=x_predict['Horizontal_Distance_To_Roadways']/7117                      
x_predict['Hillshade_9am']=x_predict['Hillshade_9am']/254                      
x_predict['Hillshade_Noon']=x_predict['Hillshade_Noon']/254                      
x_predict['Hillshade_3pm']=x_predict['Hillshade_3pm']/254                      
x_predict['Horizontal_Distance_To_Fire_Points']=x_predict['Horizontal_Distance_To_Fire_Points']/67173                      

In [ ]:
#validate data - no rows with all zeros
#x.index[x.eq(0).all(1)]
print(x[x.eq(0).all(1)].empty)
print(x_predict[x_predict.eq(0).all(1)].empty)

In [6]:
# convert the label to One Hot Encoding
#to_categorical requires 0..6 instead of 1..7
y -=1
y = y.to_numpy()

num_classes = 7

from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [7]:
#convert the features dataframes to numpy arrays
x = x.to_numpy()
x_predict = x_predict.to_numpy()

In [8]:
#split in train (80%) and test (20%) sets 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2)

In [10]:
#here is the NN model
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

num_features = 54

model = Sequential()
model.add(Dense(units=num_features, activation='relu', kernel_initializer='normal', input_dim=num_features))
model.add(Dropout(0.2))
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal'))
model.add(Dropout(0.2))
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal'))
model.add(Dropout(0.2))
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal'))
model.add(Dropout(0.2))
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
#              optimizer='Adadelta',
              optimizer='Adam',
              metrics=['accuracy'])

In [11]:
#train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=250)

Train on 12096 samples, validate on 3024 samples
Epoch 1/250
12096/12096 [==============================] - 2s 124us/sample - loss: 1.2501 - acc: 0.4377 - val_loss: 0.9161 - val_acc: 0.5939
Epoch 2/250
12096/12096 [==============================] - 1s 89us/sample - loss: 0.9538 - acc: 0.5748 - val_loss: 0.8543 - val_acc: 0.6396
Epoch 3/250
12096/12096 [==============================] - 1s 89us/sample - loss: 0.8852 - acc: 0.6049 - val_loss: 0.8288 - val_acc: 0.6366
Epoch 4/250
12096/12096 [==============================] - 1s 89us/sample - loss: 0.8525 - acc: 0.6233 - val_loss: 0.7972 - val_acc: 0.6442
Epoch 5/250
12096/12096 [==============================] - 1s 89us/sample - loss: 0.8159 - acc: 0.6538 - val_loss: 0.7780 - val_acc: 0.6822
Epoch 6/250
12096/12096 [==============================] - 1s 88us/sample - loss: 0.7916 - acc: 0.6654 - val_loss: 0.7343 - val_acc: 0.6951
Epoch 7/250
12096/12096 [==============================] - 1s 89us/sample - loss: 0.7719 - acc: 0.6782 - val_l

Epoch 59/250
12096/12096 [==============================] - 1s 89us/sample - loss: 0.5988 - acc: 0.7517 - val_loss: 0.5739 - val_acc: 0.7609
Epoch 60/250
12096/12096 [==============================] - 1s 88us/sample - loss: 0.5926 - acc: 0.7522 - val_loss: 0.5559 - val_acc: 0.7649
Epoch 61/250
12096/12096 [==============================] - 1s 87us/sample - loss: 0.5910 - acc: 0.7519 - val_loss: 0.5689 - val_acc: 0.7655
Epoch 62/250
12096/12096 [==============================] - 1s 88us/sample - loss: 0.5935 - acc: 0.7505 - val_loss: 0.5927 - val_acc: 0.7556
Epoch 63/250
12096/12096 [==============================] - 1s 88us/sample - loss: 0.5875 - acc: 0.7573 - val_loss: 0.5769 - val_acc: 0.7546
Epoch 64/250
12096/12096 [==============================] - 1s 90us/sample - loss: 0.5875 - acc: 0.7588 - val_loss: 0.5542 - val_acc: 0.7708
Epoch 65/250
12096/12096 [==============================] - 1s 88us/sample - loss: 0.5869 - acc: 0.7569 - val_loss: 0.5590 - val_acc: 0.7626
Epoch 66/250


Epoch 117/250
12096/12096 [==============================] - 1s 88us/sample - loss: 0.5409 - acc: 0.7795 - val_loss: 0.5302 - val_acc: 0.7864
Epoch 118/250
12096/12096 [==============================] - 1s 88us/sample - loss: 0.5410 - acc: 0.7759 - val_loss: 0.5326 - val_acc: 0.7827
Epoch 119/250
12096/12096 [==============================] - 1s 88us/sample - loss: 0.5428 - acc: 0.7771 - val_loss: 0.5228 - val_acc: 0.7903
Epoch 120/250
12096/12096 [==============================] - 1s 89us/sample - loss: 0.5459 - acc: 0.7758 - val_loss: 0.5371 - val_acc: 0.7765
Epoch 121/250
12096/12096 [==============================] - 1s 88us/sample - loss: 0.5388 - acc: 0.7765 - val_loss: 0.5315 - val_acc: 0.7778
Epoch 122/250
12096/12096 [==============================] - 1s 91us/sample - loss: 0.5291 - acc: 0.7822 - val_loss: 0.5253 - val_acc: 0.7847
Epoch 123/250
12096/12096 [==============================] - 1s 91us/sample - loss: 0.5336 - acc: 0.7815 - val_loss: 0.5214 - val_acc: 0.7884
Epoch 

Epoch 175/250
12096/12096 [==============================] - 1s 88us/sample - loss: 0.5006 - acc: 0.7948 - val_loss: 0.5009 - val_acc: 0.8056
Epoch 176/250
12096/12096 [==============================] - 1s 90us/sample - loss: 0.5144 - acc: 0.7935 - val_loss: 0.5113 - val_acc: 0.8036
Epoch 177/250
12096/12096 [==============================] - 1s 87us/sample - loss: 0.5102 - acc: 0.7860 - val_loss: 0.5207 - val_acc: 0.8019
Epoch 178/250
12096/12096 [==============================] - 1s 87us/sample - loss: 0.5108 - acc: 0.7924 - val_loss: 0.5074 - val_acc: 0.7943
Epoch 179/250
12096/12096 [==============================] - 1s 88us/sample - loss: 0.5096 - acc: 0.7932 - val_loss: 0.4980 - val_acc: 0.7999
Epoch 180/250
12096/12096 [==============================] - 1s 88us/sample - loss: 0.5043 - acc: 0.7917 - val_loss: 0.5093 - val_acc: 0.7983
Epoch 181/250
12096/12096 [==============================] - 1s 87us/sample - loss: 0.5043 - acc: 0.7954 - val_loss: 0.5082 - val_acc: 0.7966
Epoch 

Epoch 233/250
12096/12096 [==============================] - 1s 89us/sample - loss: 0.5031 - acc: 0.7937 - val_loss: 0.4888 - val_acc: 0.8062
Epoch 234/250
12096/12096 [==============================] - 1s 88us/sample - loss: 0.4992 - acc: 0.7944 - val_loss: 0.5077 - val_acc: 0.7900
Epoch 235/250
12096/12096 [==============================] - 1s 87us/sample - loss: 0.4864 - acc: 0.8018 - val_loss: 0.4846 - val_acc: 0.8069
Epoch 236/250
12096/12096 [==============================] - 1s 88us/sample - loss: 0.4936 - acc: 0.7996 - val_loss: 0.5020 - val_acc: 0.7943
Epoch 237/250
12096/12096 [==============================] - 1s 89us/sample - loss: 0.4931 - acc: 0.7984 - val_loss: 0.5000 - val_acc: 0.7996
Epoch 238/250
12096/12096 [==============================] - 1s 90us/sample - loss: 0.4896 - acc: 0.8032 - val_loss: 0.5191 - val_acc: 0.7923
Epoch 239/250
12096/12096 [==============================] - 1s 88us/sample - loss: 0.5004 - acc: 0.7966 - val_loss: 0.4924 - val_acc: 0.8069
Epoch 

In [12]:
# Predict!!
y_predict = model.predict(x_predict)

In [ ]:
for i in range(10):
	print(y_predict[i], np.argmax(y_predict[i])+1)

In [13]:
# Save predictions to a file for submission
#argmax give us the highest probable label
# we add one to the predictions to scale from 0..6 to 1..7
output = pd.DataFrame({'Id': Ids,
                       'Cover_Type': y_predict.argmax(axis=1)+1})
output.to_csv('submission.csv', index=False)

In [14]:
#create a link to download the file    
from IPython.display import FileLink
FileLink(r'submission.csv')

/kaggle/working/submission.csv